In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

print("Environment ready for Anime Recommender 🚀")


Environment ready for Anime Recommender 🚀


In [4]:
anime_df = pd.read_csv("anime_standalone.csv")
anime_df.head()

,id,title,titleJa,titleEn,image,mean,rank,num_list_users,num_scoring_users,num_episodes,start_date,end_date,media_type,status,rating,average_episode_duration,genres,studios
0,1,Cowboy Bebop,カウボーイビバップ,Cowboy Bebop,https://cdn.myanimelist.net/images/anime/4/196...,8.75,48.0,1983092,1023611,26,1998-04-03,1999-04-24,tv,finished_airing,r,1440,"Action, Adult Cast, Award Winning, Sci-Fi, Space",Sunrise
1,5,Cowboy Bebop: Tengoku no Tobira,カウボーイビバップ 天国の扉,Cowboy Bebop: The Movie,https://cdn.myanimelist.net/images/anime/1439/...,8.38,217.0,399298,225988,1,2001-09-01,2001-09-01,movie,finished_airing,r,6911,"Action, Adult Cast, Sci-Fi, Space",Bones
2,6,Trigun,トライガン,Trigun,https://cdn.myanimelist.net/images/anime/1130/...,8.22,387.0,805871,390093,26,1998-04-01,1998-09-30,tv,finished_airing,pg_13,1480,"Action, Adult Cast, Adventure, Sci-Fi, Shounen",Madhouse
3,7,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),Witch Hunter Robin,https://cdn.myanimelist.net/images/anime/10/19...,7.23,3286.0,123958,45437,26,2002-07-03,2002-12-25,tv,finished_airing,pg_13,1500,"Action, Detective, Drama, Mystery, Supernatural",Sunrise
4,8,Bouken Ou Beet,冒険王ビィト,Beet the Vandel Buster,https://cdn.myanimelist.net/images/anime/7/215...,6.92,4834.0,16334,6970,52,2004-09-30,2005-09-29,tv,finished_airing,pg,1380,"Action, Adventure, Fantasy, Shounen",Toei Animation


In [5]:
# Step 3: Clean the data for safe processing
# Fill missing genres with empty string
anime_df['genres'] = anime_df['genres'].fillna('')

# Print how many animes we have
print("Total animes in our robot's brain:", len(anime_df))
anime_df[['title', 'genres']].head()


Total animes in our robot's brain: 23694


,title,genres
0,Cowboy Bebop,"Action, Adult Cast, Award Winning, Sci-Fi, Space"
1,Cowboy Bebop: Tengoku no Tobira,"Action, Adult Cast, Sci-Fi, Space"
2,Trigun,"Action, Adult Cast, Adventure, Sci-Fi, Shounen"
3,Witch Hunter Robin,"Action, Detective, Drama, Mystery, Supernatural"
4,Bouken Ou Beet,"Action, Adventure, Fantasy, Shounen"


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Step 4: Convert genres to vectors (numbers) using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
genre_vectors = vectorizer.fit_transform(anime_df['genres'])


In [7]:
# Step 5: Calculate similarity between all anime
similarity_scores = cosine_similarity(genre_vectors)


In [12]:
# Step 6: Define a function that recommends anime
def recommend_anime(title, top_n=5):
    # Find index of the anime by title
    idx = anime_df[anime_df['title'].str.lower() == title.lower()].index

    if len(idx) == 0:
        print("❌ Anime not found. Please check spelling.")
        return

    idx = idx[0]

    # Get scores for that anime with all others
    sim_scores = list(enumerate(similarity_scores[idx]))

    # Sort by score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get top N similar anime (excluding the input itself)
    print(f"\n📺 If you liked '{title}', you might also like:\n")
    count = 0
    for i in sim_scores[1:]:
        recommended_title = anime_df.iloc[i[0]]['title']
        print(f"- {recommended_title}")
        count += 1
        if count >= top_n:
            break


In [13]:
recommend_anime("Naruto")




📺 If you liked 'Naruto', you might also like:

- Naruto: Takigakure no Shitou - Ore ga Eiyuu Dattebayo!
- Naruto: Shippuuden
- Juushin Enbu: Hero Tales
- Dragon Ball Z Movie 14: Kami to Kami
- Boruto: Naruto the Movie


In [18]:
# Shows matching titles if the anime is not found
def suggest_titles(input_title):
    input_title = input_title.lower()
    matches = anime_df[anime_df['title'].str.lower().str.contains(input_title)]
    if matches.empty:
        print("No similar titles found.")
    else:
        print("\nDid you mean:")
        for title in matches['title'].head(5):
            print(">", title)


In [19]:
suggest_titles("Naruto")


Did you mean:
> Naruto
> Naruto Movie 1: Dai Katsugeki!! Yuki Hime Ninpouchou Dattebayo!
> Naruto: Takigakure no Shitou - Ore ga Eiyuu Dattebayo!
> Naruto: Akaki Yotsuba no Clover wo Sagase
> Naruto Movie 2: Dai Gekitotsu! Maboroshi no Chiteiiseki Dattebayo!
